In [24]:
import torch

In [25]:
#devito set up
from abc import ABC, abstractmethod
from devito import Operator, Function
from numpy import array
import numpy as np
from devito import Grid, Function, dimensions, Eq, Inc
import sympy
class Layer(ABC):
    def __init__(self, input_data):
        self._input_data = input_data
        self._R = self._allocate()

    @abstractmethod
    def _allocate(self) -> Function:
        # This method should return a Function object corresponding to
        # an output of the layer.
        pass

    def execute(self) -> (Operator, array):
        op = Operator(self.equations())
        op.cfunction

        return (op, self._R.data)

    @abstractmethod
    def equations(self) -> list:
        pass

Plain 2D conv 

In [95]:
single_image = torch.randint(0,60,(1,3,3,3))
class Subsampling(Layer):
    def __init__(self, kernel_size, feature_map, function,
                 stride=(1, 1), padding=(0, 0), activation=None,
                 bias=0):
        # All sizes are expressed as (rows, columns).

        self._error_check(kernel_size, feature_map, stride, padding)

        self._kernel_size = kernel_size
        self._function = function
        self._activation = activation
        self._bias = bias

        self._stride = stride
        self._padding = padding

        super().__init__(input_data=feature_map)

    def _error_check(self, kernel_size, feature_map, stride, padding):
        if feature_map is None or len(feature_map) == 0:
            raise Exception("Feature map must not be empty")

        if kernel_size is None or len(kernel_size) != 2:
            raise Exception("Kernel size is incorrect")

        if stride is None or len(stride) != 2:
            raise Exception("Stride is incorrect")

        if stride[0] < 1 or stride[1] < 1:
            raise Exception("Stride cannot be less than 1")

        if padding is None or len(padding) != 2:
            raise Exception("Padding is incorrect")

        if padding[0] < 0 or padding[1] < 0:
            raise Exception("Padding cannot be negative")

        map_height = len(feature_map) + 2 * padding[0]
        map_width = len(feature_map[0]) + 2 * padding[1]
        kernel_height, kernel_width = kernel_size

        if (map_height - kernel_height) % stride[0] != 0 or \
           (map_width - kernel_width) % stride[1] != 0:
            raise Exception("Stride " + str(stride) + " is not "
                            "compatible with feature map, kernel and padding "
                            "sizes")

    def _allocate(self):
        map_height = len(self._input_data) + 2 * self._padding[0]
        map_width = len(self._input_data[0]) + 2 * self._padding[1]
        kernel_height, kernel_width = self._kernel_size

        gridB = Grid(shape=(map_height, map_width))
        B = Function(name='B', grid=gridB, space_order=0)

        a, b = dimensions('a b')
        gridR = Grid(shape=((map_height - kernel_height + self._stride[0])
                            // self._stride[0],
                            (map_width - kernel_width + self._stride[1])
                            // self._stride[1]),
                     dimensions=(a, b))
        R = Function(name='R', grid=gridR, space_order=0)

        for i in range(self._padding[0], map_height - self._padding[0]):
            B.data[i] = \
                np.concatenate(([0] * self._padding[1],
                                self._input_data[i - self._padding[0]],
                                [0] * self._padding[1]))

        self._B = B
        return R

    def equations(self):
        a, b = self._B.dimensions
        kernel_height, kernel_width = self._kernel_size

        rhs = self._function([self._B[self._stride[0] * a + i,
                                      self._stride[1] * b + j]
                              for i in range(kernel_height)
                              for j in range(kernel_width)]) + self._bias
        print("tyoe=",type(rhs))
        if self._activation is not None:
            rhs = self._activation(rhs)

        return [Eq(self._R[a, b], rhs)]
                
Sample_2D = Subsampling((2,2),single_image[0][0], lambda l: sympy.Max(*l))
tup_2d = Sample_2D.execute()
tup_2d[0].apply()
tup_2d[1].shape

Operator `Kernel` run in 0.01 s


tyoe= Max


(2, 2)

In [55]:
Sample_2D.equations()

[Eq(R[x, y], Max(B[x, y], B[x, y + 1], B[x + 1, y], B[x + 1, y + 1]))]

Now the convolution in 4D - Naively

In [60]:
class Subsampling_4d(Layer):
    def __init__(self, kernel_size, feature_map, function,
                 stride=(1, 1), padding=(0, 0), activation=None,
                 bias=0):
        # All sizes are expressed as (rows, columns).

        #self._error_check(kernel_size, feature_map, stride, padding)

        self._kernel_size = kernel_size
        self._function = function
        self._activation = activation
        self._bias = bias

        self._stride = stride
        self._padding = padding

        super().__init__(input_data=feature_map)


    def _allocate(self):
        map_height = self._input_data.shape[2] + 2 * self._padding[0]
        map_width = self._input_data.shape[3] + 2 * self._padding[1]
        kernel_height, kernel_width = self._kernel_size
        a, b, c, d = dimensions('a b c d')
        gridB = Grid(shape=(self._input_data.shape[0], self._input_data.shape[1], map_height, map_width),\
                    dimensions=(a, b, c, d))
        B = Function(name='B', grid=gridB, space_order=0)

        e, f, g, h = dimensions('e f g h')
        gridR = Grid(shape=( self._input_data.shape[0],  self._input_data.shape[1],\
                            (map_height - kernel_height + self._stride[0])
                            // self._stride[0],
                            (map_width - kernel_width + self._stride[1])
                            // self._stride[1]),
                     dimensions=(e, f, g, h))
        print(gridR)
        R = Function(name='R', grid=gridR, space_order=0)
        #add padding to start and end of each row
        for image in range(self._input_data.shape[0]):
            for channel in range(self._input_data.shape[1]):
                for i in range(self._padding[0], map_height - self._padding[0]):
                    B.data[image, channel, i] = \
                        np.concatenate(([0] * self._padding[1],
                                        self._input_data[image, channel, i - self._padding[0]],
                                        [0] * self._padding[1]))

        self._B = B
        return R

    def equations(self):
        a, b, c, d = self._B.dimensions
        kernel_height, kernel_width = self._kernel_size
        images = self._input_data.shape[0]
        channels = self._input_data.shape[1] 
        rhs = self._function([self._B[image, channel, self._stride[0] * c + i,
                                      self._stride[1] * d + j]
                              for image in range(images)
                              for channel in range(channels)
                              for i in range(kernel_height)
                              for j in range(kernel_width)
                              ]) + self._bias

        print("rhs:", rhs)
        print("type:", type(rhs))
        if self._activation is not None:
            rhs = self._activation(rhs)

        return [Eq(self._R[a, b, c, d], rhs)]

Sample_obj4d = Subsampling_4d((2,2),single_image, lambda l: sympy.Max(*l))
tup4d = Sample_obj4d.execute()
tup4d[0].apply()
tup4d[1].shape

/app/devito/types/grid.py:206: RuntimeWarning: divide by zero encountered in true_divide
  spacing = (np.array(self.extent) / (np.array(self.shape) - 1)).astype(self.dtype)


Grid[extent=(1.0, 1.0, 1.0, 1.0), shape=(1, 3, 2, 2), dimensions=(e, f, g, h)]
rhs: Max(B[0, 0, c, d], B[0, 0, c, d + 1], B[0, 0, c + 1, d], B[0, 0, c + 1, d + 1], B[0, 1, c, d], B[0, 1, c, d + 1], B[0, 1, c + 1, d], B[0, 1, c + 1, d + 1], B[0, 2, c, d], B[0, 2, c, d + 1], B[0, 2, c + 1, d], B[0, 2, c + 1, d + 1])
type: Max


Operator `Kernel` run in 0.01 s


(1, 3, 2, 2)

In [63]:
tup4d[1]

Data([[[[59., 55.],
        [59., 58.]],

       [[59., 55.],
        [59., 58.]],

       [[59., 55.],
        [59., 58.]]]], dtype=float32)

We can see that we are not getting what we expect.
The operator should have a Max for each channel and each image.
Also in the result, are getting the biggest values across all 3 channels

A less naive implementation, but it results in list/generator

In [65]:
class Subsampling_4d(Layer):
    def __init__(self, kernel_size, feature_map, function,
                 stride=(1, 1), padding=(0, 0), activation=None,
                 bias=0):
        # All sizes are expressed as (rows, columns).

        #self._error_check(kernel_size, feature_map, stride, padding)

        self._kernel_size = kernel_size
        self._function = function
        self._activation = activation
        self._bias = bias

        self._stride = stride
        self._padding = padding

        super().__init__(input_data=feature_map)


    def _allocate(self):
        map_height = self._input_data.shape[2] + 2 * self._padding[0]
        map_width = self._input_data.shape[3] + 2 * self._padding[1]
        kernel_height, kernel_width = self._kernel_size
        a, b, c, d = dimensions('a b c d')
        gridB = Grid(shape=(self._input_data.shape[0], self._input_data.shape[1], map_height, map_width),\
                    dimensions=(a, b, c, d))
        B = Function(name='B', grid=gridB, space_order=0)

        e, f, g, h = dimensions('e f g h')
        gridR = Grid(shape=( self._input_data.shape[0],  self._input_data.shape[1],\
                            (map_height - kernel_height + self._stride[0])
                            // self._stride[0],
                            (map_width - kernel_width + self._stride[1])
                            // self._stride[1]),
                     dimensions=(e, f, g, h))
        print(gridR)
        R = Function(name='R', grid=gridR, space_order=0)
        #add padding to start and end of each row
        for image in range(self._input_data.shape[0]):
            for channel in range(self._input_data.shape[1]):
                for i in range(self._padding[0], map_height - self._padding[0]):
                    B.data[image, channel, i] = \
                        np.concatenate(([0] * self._padding[1],
                                        self._input_data[image, channel, i - self._padding[0]],
                                        [0] * self._padding[1]))

        self._B = B
        return R

    def equations(self):
        a, b, c, d = self._B.dimensions
        kernel_height, kernel_width = self._kernel_size
        images = self._input_data.shape[0]
        channels = self._input_data.shape[1] 
        rhs = ([self._function([self._B[image, channel, self._stride[0] * c + i,
                                      self._stride[1] * d + j]
                              for i in range(kernel_height)
                              for j in range(kernel_width)])
                              for channel in range(channels)
                            for image in range(images)])
        print("rhs:", rhs)
        print("type:", type(rhs))
        if self._activation is not None:
            rhs = self._activation(rhs)

        return [Eq(self._R[a, b, c, d], rhs)]

Sample_obj4 = Subsampling_4d((2,2),single_image, lambda l: sympy.Max(*l))
#A.equations()
tup4 = Sample_obj4.execute()
tup4[0].apply()
tup4[1].shape

Grid[extent=(1.0, 1.0, 1.0, 1.0), shape=(1, 3, 2, 2), dimensions=(e, f, g, h)]
rhs: [Max(B[0, 0, c, d], B[0, 0, c, d + 1], B[0, 0, c + 1, d], B[0, 0, c + 1, d + 1]), Max(B[0, 1, c, d], B[0, 1, c, d + 1], B[0, 1, c + 1, d], B[0, 1, c + 1, d + 1]), Max(B[0, 2, c, d], B[0, 2, c, d + 1], B[0, 2, c + 1, d], B[0, 2, c + 1, d + 1])]
type: <class 'list'>


/app/devito/types/grid.py:206: RuntimeWarning: divide by zero encountered in true_divide
  spacing = (np.array(self.extent) / (np.array(self.shape) - 1)).astype(self.dtype)


SympifyError: SympifyError: [Max(B[0, 0, c, d], B[0, 0, c, d + 1], B[0, 0, c + 1, d], B[0, 0, c + 1, d + 1]), Max(B[0, 1, c, d], B[0, 1, c, d + 1], B[0, 1, c + 1, d], B[0, 1, c + 1, d + 1]), Max(B[0, 2, c, d], B[0, 2, c, d + 1], B[0, 2, c + 1, d], B[0, 2, c + 1, d + 1])]

The function should be [Max(B[**a, b**, c, d] instead of [Max(B[0, 0, c, d]

Doing it the inefficent way:

In [66]:
Sample_2D = Subsampling((2,2),single_image[0][0], lambda l: sympy.Max(*l))
tup_2d = Sample_2D.execute()
tup_2d[0].apply()
tup_2d[1].shape

Operator `Kernel` run in 0.01 s


(2, 2)

In [88]:
results_tensor = torch.ones(1,3,2,2)
for channel in range(single_image.shape[1]):
    Sample_2D_channel = Subsampling((2,2),single_image[0][channel], lambda l: sympy.Max(*l))
    tup_2d = Sample_2D_channel.execute()
    tup_2d[0].apply()
    results_tensor[0][channel] = torch.from_numpy(tup_2d[1])

Operator `Kernel` run in 0.01 s
Operator `Kernel` run in 0.01 s
Operator `Kernel` run in 0.01 s


In [89]:
results_tensor

tensor([[[[59., 55.],
          [59., 55.]],

         [[32., 32.],
          [59., 58.]],

         [[54., 31.],
          [54., 22.]]]])

This works but we have to call the opperator as many times as the channels and the images

3d max sampling, only channels

In [105]:
class Subsampling(Layer):
    def __init__(self, kernel_size, feature_map, function,
                 stride=(1, 1), padding=(0, 0), activation=None,
                 bias=0):
        

        self._kernel_size = kernel_size
        self._function = function
        self._activation = activation
        self._bias = bias

        self._stride = stride
        self._padding = padding

        super().__init__(input_data=feature_map)

   

    def _allocate(self):
        map_height = self._input_data.shape[1] + 2 * self._padding[0]
        map_width = self._input_data.shape[2] + 2 * self._padding[1]
        kernel_height, kernel_width = self._kernel_size
        b, c, d = dimensions('b c d')
        gridB = Grid(shape=(self._input_data.shape[0], map_height, map_width),\
                    dimensions=(b, c, d))
        B = Function(name='B', grid=gridB, space_order=0)

        f, g, h = dimensions('f g h')
        gridR = Grid(shape=( self._input_data.shape[0],\
                            (map_height - kernel_height + self._stride[0])
                            // self._stride[0],
                            (map_width - kernel_width + self._stride[1])
                            // self._stride[1]),
                     dimensions=(f, g, h))
        print(gridR)
        R = Function(name='R', grid=gridR, space_order=0)
        #add padding to start and end of each row
        for channel in range(self._input_data.shape[0]):
            for i in range(self._padding[0], map_height - self._padding[0]):
                B.data[channel, i] = \
                    np.concatenate(([0] * self._padding[1],
                                    self._input_data[channel, i - self._padding[0]],
                                    [0] * self._padding[1]))

        self._B = B
        return R
    def equations(self):
        a, b, c = self._B.dimensions
        kernel_height, kernel_width = self._kernel_size
        channels = self._input_data.shape[0]
        rhs = self._function([self._B[channel, self._stride[0] * b + i,
                                      self._stride[1] * c + j]
                              for i in range(kernel_height)
                              for j in range(kernel_width)
                              for channel in range(channels)) + self._bias
        print("type=",type(rhs))
        print("rhs:", rhs)
        if self._activation is not None:
            rhs = self._activation(rhs)

        return [Eq(self._R[a, b, c], rhs)]
                
Sample_2D = Subsampling((2,2),single_image[0], lambda l: sympy.Max(*l))
tup_2d = Sample_2D.execute()
tup_2d[0].apply()
tup_2d[1].shape

Grid[extent=(1.0, 1.0, 1.0), shape=(3, 2, 2), dimensions=(f, g, h)]
type= Max
rhs: Max(B[0, c, d], B[0, c, d + 1], B[0, c + 1, d], B[0, c + 1, d + 1], B[1, c, d], B[1, c, d + 1], B[1, c + 1, d], B[1, c + 1, d + 1], B[2, c, d], B[2, c, d + 1], B[2, c + 1, d], B[2, c + 1, d + 1])


Operator `Kernel` run in 0.01 s


(3, 2, 2)

In [104]:
tup_2d[1]

Data([[[57., 58.],
       [55., 58.]],

      [[57., 58.],
       [55., 58.]],

      [[57., 58.],
       [55., 58.]]], dtype=float32)